In [17]:
# Importing Libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
import sklearn
import yake
import spacy

%matplotlib inline



In [16]:


# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install spacy

# python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.5 MB 557 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 450 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 7.1 MB/s  eta 0:00:01
     |████████████████████████████████| 105 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 4.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=f68d66d3e2ce8d0f04de74cfe70217618e7cffe9268ee56e71a86fd9aeedfaa8
  Stored in directory: /Users/andronikos/Library/Caches/pip/wheels/11/73/9a/f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open
  Att

In [7]:
LocationTrue = r'./True.csv'
LocationFake = r'./Fake.csv'

dfTrue = pd.read_csv(LocationTrue)
dfFake = pd.read_csv(LocationFake)

### Preprocessing

In [6]:
def preprocessing(dataframe):
    symbols = "[!\"#$%&()*+-./:;<=>?@[\]^_`{|}'~\n]"
    
    dataframe['title'] = dataframe['title'].str.lower()
#     dataframe['title'] = dataframe['title'].str.replace(symbols, '', regex=True)
    dataframe['text'] = dataframe['text'].str.lower()
#     dataframe['text'] = dataframe['text'].str.replace(symbols, '', regex=True)

preprocessing(dfTrue)
preprocessing(dfFake)
dfTrue

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [10]:
titles = dfTrue['title'].tolist()
texts = dfTrue['text'].tolist()

dictList = [dict() for x in range(len(titles))]


# tf_idf = {}
# for i in range(N):
#     tokens = processed_text[i]
#     counter = Counter(tokens + processed_title[i])
#     for token in np.unique(tokens):
#         tf = counter[token]/words_count
#         df = doc_freq(token)
#         idf = np.log(N/(df+1))
#         tf_idf[doc, token] = tf*idf


for i in range(len(titles)):
    i = 1

In [14]:
kw_extractor = yake.KeywordExtractor()
# text = """spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython. The library is published under the MIT license and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
text = titles[0]
language = "en"
max_ngram_size = 2
deduplication_threshold = 0.1
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

('Republicans flip', 0.026233073037508336)
('fight looms', 0.04940384002065631)
('budget', 0.15831692877998726)


In [19]:
nlp = spacy.load("en_core_web_sm")
# text = """spaCy is an open-source software library for advanced natural language processing, 
# written in the programming languages Python and Cython. The library is published under the MIT license
# and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
doc = nlp(text)
print(doc.ents)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (2, 2)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text])
candidates = count.get_feature_names()
print(candidates)

['budget fight', 'fight looms', 'fiscal script', 'flip fiscal', 'looms republicans', 'republicans flip']


<!-- https://lena-voita.github.io/nlp_course/word_embeddings.html
https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb
https://web.stanford.edu/~jurafsky/slp3/slides/LM_4.pdf -->

https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea